**Индивидуальное задание**

1. Набор данных zoo.data.
2. Выполните построение модели классификации на основе метода ближайших соседей. В ходе решения задачи необходимо решить следующие подзадачи: 2.1 Построение классификатора с заданием K (количества ближайших соседей) пользователем; 2.2 Вычисление оценки hold-out для различнх значений K, а также для различных долей обучающей и тестирующей подвыборок; 2.3 Вычисление оценки cross validation для различных значений K, а также для различных значений fold (количества подмножеств при кросс- валидации). 2.4 Вычислите оптимальные значения K. Обоснуйте свой выбор. Продемонстрируйте использование полученного классификатора.

**2.1 Построение классификатора KNN с заданным 𝐾**

In [14]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, classification_report

# Загрузка данных зоопарка
data = pd.read_csv('zoo.data', header=None)
data.columns = ['animal_name', 'hair', 'feathers', 'eggs', 'milk', 'airborne', 
                'aquatic', 'predator', 'toothed', 'backbone', 'breathes', 
                'venomous', 'fins', 'legs', 'tail', 'domestic', 'catsize', 'class_type']

# Разделение признаков и целевой переменной
X = data.iloc[:, 1:-1]
y = data.iloc[:, -1]

# Функция для оценки KNN с разными параметрами
def evaluate_knn(X, y, k_range=range(1, 21), test_sizes=[0.2, 0.3], cv_folds=[5, 10]):
    # Словарь для хранения результатов
    results = {'hold_out': {}, 'cross_val': {}}
    
    # Масштабирование признаков
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)
    
    # Валидация с удержанием
    for test_size in test_sizes:
        results['hold_out'][test_size] = {}
        for k in k_range:
            # Разделение данных
            X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=test_size)
            
            # Обучение и оценка
            knn = KNeighborsClassifier(n_neighbors=k)
            knn.fit(X_train, y_train)
            y_pred = knn.predict(X_test)
            results['hold_out'][test_size][k] = accuracy_score(y_test, y_pred)
    
    # Кросс-валидация
    for fold in cv_folds:
        results['cross_val'][fold] = {}
        for k in k_range:
            knn = KNeighborsClassifier(n_neighbors=k)
            scores = cross_val_score(knn, X_scaled, y, cv=fold)
            results['cross_val'][fold][k] = scores.mean()
    
    return results

# Выполнение оценки
results = evaluate_knn(X, y)

# Функция для поиска оптимального K
def find_optimal_k(results):
    # Анализ результатов кросс-валидации
    cv_results = results['cross_val']
    best_overall = {}
    
    for fold, k_scores in cv_results.items():
        best_k = max(k_scores, key=k_scores.get)
        best_score = k_scores[best_k]
        best_overall[fold] = (best_k, best_score)
    
    return best_overall

# Получение оптимального K
optimal_k = find_optimal_k(results)

# Вывод результатов
print("Hold-Out Validation Results:")
for test_size, k_scores in results['hold_out'].items():
    print(f"\nTest Size: {test_size}")
    for k, score in sorted(k_scores.items(), key=lambda x: x[1], reverse=True)[:5]:
        print(f"K={k}: {score:.4f}")

print("\nCross-Validation Results:")
for fold, k_scores in results['cross_val'].items():
    print(f"\nFold: {fold}")
    for k, score in sorted(k_scores.items(), key=lambda x: x[1], reverse=True)[:5]:
        print(f"K={k}: {score:.4f}")

print("\nOptimal K for Each Fold:")
for fold, (k, score) in optimal_k.items():
    print(f"Fold {fold}: K={k}, Score={score:.4f}")

# Финальная модель с лучшим K
best_k = max(optimal_k, key=lambda x: optimal_k[x][1])
best_k_value = optimal_k[best_k][0]
X_scaled = StandardScaler().fit_transform(X)
final_knn = KNeighborsClassifier(n_neighbors=best_k_value)
final_knn.fit(X_scaled, y)

# Пример прогноза
print("\nSample Prediction:")
sample_animal = X.iloc[0].values.reshape(1, -1)
sample_scaled = StandardScaler().fit(X).transform(sample_animal)
prediction = final_knn.predict(sample_scaled)
print(f"Sample Animal Features: {dict(X.iloc[0])}")
print(f"Predicted Class: {prediction[0]}")

C:\Users\SanchoKusika\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\model_selection\_split.py:776: UserWarning: The least populated class in y has only 4 members, which is less than n_splits=5.
  warnings.warn(
C:\Users\SanchoKusika\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\model_selection\_split.py:776: UserWarning: The least populated class in y has only 4 members, which is less than n_splits=5.
  warnings.warn(
C:\Users\SanchoKusika\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\model_selection\_split.py:776: UserWarning: The least populated class in y has only 4 members, which is less than n_splits=5.
  warnings.warn(
C:\Users\SanchoKusika\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\model_selection\_split.py:776: UserWarning: The least populated class in y has only 4 members, which is less than n_splits=5.
  warnings.warn(
C:\Users\SanchoKusika\AppData\Local\Programs\Python\Python312\Lib\site-packa

Hold-Out Validation Results:

Test Size: 0.2
K=4: 1.0000
K=2: 0.9524
K=3: 0.9524
K=5: 0.9524
K=12: 0.9524

Test Size: 0.3
K=3: 0.9677
K=4: 0.9677
K=1: 0.9355
K=11: 0.9355
K=5: 0.9032

Cross-Validation Results:

Fold: 5
K=1: 0.9505
K=2: 0.9400
K=5: 0.9400
K=3: 0.9305
K=6: 0.9300

Fold: 10
K=1: 0.9509
K=2: 0.9400
K=5: 0.9400
K=3: 0.9300
K=6: 0.9200

Optimal K for Each Fold:
Fold 5: K=1, Score=0.9505
Fold 10: K=1, Score=0.9509

Sample Prediction:
Sample Animal Features: {'hair': np.int64(1), 'feathers': np.int64(0), 'eggs': np.int64(0), 'milk': np.int64(1), 'airborne': np.int64(0), 'aquatic': np.int64(0), 'predator': np.int64(1), 'toothed': np.int64(1), 'backbone': np.int64(1), 'breathes': np.int64(1), 'venomous': np.int64(0), 'fins': np.int64(0), 'legs': np.int64(4), 'tail': np.int64(0), 'domestic': np.int64(0), 'catsize': np.int64(1)}
Predicted Class: 1


C:\Users\SanchoKusika\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


**Выводы:**
1. Модель показывает высокую точность (95-100%) с малыми значениями K (1-2)
2. Оптимальное количество соседей K = 1
3. Признаки животных хорошо разделены в пространстве признаков
4. Метод ближайших соседей эффективен для данного набора данных